In [1]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import numpy as np
from PIL import Image
from datasets import load_dataset
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType
from pyspark.sql import SparkSession
import io

c:\Users\marco\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

In [4]:
sc._conf.getAll()

[('spark.sql.warehouse.dir',
  'file:/C:/Users/marco/Desktop/project_bdc/spark-warehouse'),
 ('spark.app.name', 'PySparkTutorial'),
 ('spark.driver.port', '62021'),
 ('spark.driver.host', 'LAPTOP-JLLVBEPM'),
 ('spark.driver.memory', '45G'),
 ('spark.app.startTime', '1682852096077'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.maxResultSize', '10G'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-op

In [178]:
#open csv file
df_appearances = spark.read.csv("archive/appearances.csv", header=True, inferSchema=True)
df_club_games = spark.read.csv("archive/club_games.csv", header=True, inferSchema=True)
df_clubs = spark.read.csv("archive/clubs.csv", header=True, inferSchema=True)
df_competitions = spark.read.csv("archive/competitions.csv", header=True, inferSchema=True)
df_game_events = spark.read.csv("archive/game_events.csv", header=True, inferSchema=True)
df_games = spark.read.csv("archive/games.csv", header=True, inferSchema=True)
df_player_valutations = spark.read.csv("archive/player_valuations.csv", header=True, inferSchema=True)
df_players = spark.read.csv("archive/players.csv", header=True, inferSchema=True)

In [176]:
#print number of rows for each dataset
print("Number of rows: ", df_appearances.count())
print("Number of rows: ", df_club_games.count())
print("Number of rows: ", df_clubs.count())
print("Number of rows: ", df_competitions.count())
print("Number of rows: ", df_game_events.count())
print("Number of rows: ", df_games.count())
print("Number of rows: ", df_player_valutations.count())
print("Number of rows: ", df_players.count())

df_appearances.printSchema()
df_game_events.printSchema()
df_games.printSchema()

df_appearances.show()
df_game_events.show()
df_games.show()


Number of rows:  1166215
Number of rows:  123096
Number of rows:  411
Number of rows:  43
Number of rows:  559853
Number of rows:  61548
Number of rows:  214925
Number of rows:  28503
root
 |-- appearance_id: string (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- player_current_club_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- player_name: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)

root
 |-- game_id: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- club_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- description: string (nullable = true)
 

1 - unire players e appearances
2 - aggiungere la media punti per partita dei club per cui ha giocato in quell'anno
3 - aggiungere l'ultimo valore di mercato ottenuto in anno (player_valuations)

PRENDERE IN CONSIDERAZIONE LA POSSIBILITà DI ELIMINARE TOTALMENTE QUELLO CHE è SUCCESSO NEL 2023.

In [148]:
# join players and appearances
df_players_appearances = df_players.join(df_appearances, ["player_id"], how='inner')

In [149]:
df_players_appearances.show()

+---------+--------------------+---------------+-----------------+----------------------+-----------------+-------------+-------------+--------+--------------+-----+------------+-------------------+---------------------------+----------+------------------------+------------------------------------+----------+------------+--------------------+--------------------+-----------+--------------------+-------------+-------+--------------+----------------------+----------+--------------------+--------------+------------+---------+-----+-------+--------------+
|player_id|                name|current_club_id|current_club_name|country_of_citizenship| country_of_birth|city_of_birth|date_of_birth|position|  sub_position| foot|height_in_cm|market_value_in_eur|highest_market_value_in_eur|agent_name|contract_expiration_date|current_club_domestic_competition_id|first_name|   last_name|         player_code|           image_url|last_season|                 url|appearance_id|game_id|player_club_id|player_

In [150]:
#aggiungere a df_players_appearances la colonna year che contiene solo l'anno della feature date
from pyspark.sql.functions import year

df_players_appearances = df_players_appearances.withColumn("year", year(df_players_appearances.date))

In [151]:
df_players_appearances.show()

+---------+--------------------+---------------+-----------------+----------------------+-----------------+-------------+-------------+--------+--------------+-----+------------+-------------------+---------------------------+----------+------------------------+------------------------------------+----------+------------+--------------------+--------------------+-----------+--------------------+-------------+-------+--------------+----------------------+----------+--------------------+--------------+------------+---------+-----+-------+--------------+----+
|player_id|                name|current_club_id|current_club_name|country_of_citizenship| country_of_birth|city_of_birth|date_of_birth|position|  sub_position| foot|height_in_cm|market_value_in_eur|highest_market_value_in_eur|agent_name|contract_expiration_date|current_club_domestic_competition_id|first_name|   last_name|         player_code|           image_url|last_season|                 url|appearance_id|game_id|player_club_id|pl

In [152]:
# drop da df_players_appearances le colonne current_club_id, current_club_name, city_of_birth, market_value_in_eur, contract_expiration_date, agent_name, current_club_domestic_competition_id, image_url, last_season, url, game_id, player_current_club_id
df_players_appearances = df_players_appearances.drop("current_club_id", "date", "appearance_id", "current_club_name", "city_of_birth", "market_value_in_eur", "contract_expiration_date", "agent_name", "current_club_domestic_competition_id", "image_url", "last_season", "url", "game_id", "player_current_club_id")
# in forse: highest_market_value_in_eur

In [153]:
df_players_appearances.show()

+---------+--------------------+----------------------+-----------------+-------------+--------+--------------+-----+------------+---------------------------+----------+------------+--------------------+--------------+--------------------+--------------+------------+---------+-----+-------+--------------+----+
|player_id|                name|country_of_citizenship| country_of_birth|date_of_birth|position|  sub_position| foot|height_in_cm|highest_market_value_in_eur|first_name|   last_name|         player_code|player_club_id|         player_name|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|
+---------+--------------------+----------------------+-----------------+-------------+--------+--------------+-----+------------+---------------------------+----------+------------+--------------------+--------------+--------------------+--------------+------------+---------+-----+-------+--------------+----+
|    52453|       Haris Handzic|    Bosnia-Herzegovina|Jugoslawi

In [169]:
#TEST
print(df_players_appearances.filter(df_players_appearances.player_id == 148455).count())
df_players_appearances.filter(df_players_appearances.player_id == 148455).agg(sum(col("goals"))).show()

414
+----------+
|sum(goals)|
+----------+
|       227|
+----------+



In [159]:

from pyspark.sql.functions import sum, col

df_new = df_players_appearances.groupBy("player_id", "competition_id", "year", "player_club_id").agg(
    sum(col("yellow_cards")).alias("yellow_cards"),
    sum(col("red_cards")).alias("red_cards"),
    sum(col("goals")).alias("goals"),
    sum(col("assists")).alias("assists"),
    sum(col("minutes_played")).alias("minutes_played")
).select("player_id", "competition_id", "year", "player_club_id", "yellow_cards", "red_cards", "goals", "assists", "minutes_played").distinct()


In [160]:
df_new.show()

+---------+--------------+----+--------------+------------+---------+-----+-------+--------------+
|player_id|competition_id|year|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|
+---------+--------------+----+--------------+------------+---------+-----+-------+--------------+
|    16137|           IT1|2014|          1429|           1|        0|    0|      1|           736|
|   175589|           SC1|2015|          2999|           0|        0|    0|      0|           607|
|    35183|            L1|2014|            24|           5|        0|    4|      3|           818|
|    58358|           DFB|2014|            27|           0|        0|    1|      1|           180|
|    47571|           ELQ|2014|          7769|           0|        0|    0|      0|           227|
|   217839|           GR1|2014|           128|           1|        0|    0|      1|           221|
|    87423|           GRP|2015|          5219|           0|        0|    0|      0|            90|
|    56786

In [168]:
#TEST
print(df_new.filter(df_new.player_id == 148455).count())
df_new.filter(df_new.player_id == 148455).agg(sum(col("goals"))).show()

46
+----------+
|sum(goals)|
+----------+
|       227|
+----------+



In [164]:
from pyspark.sql.functions import collect_set, sum

# Raggruppare per (player_id, year) e raccogliere i valori unici di competition_id e player_club_id in liste
df_grouped = df_new.groupBy("player_id", "year") \
              .agg(collect_set("competition_id").alias("competition_id_list"),
                   collect_set("player_club_id").alias("player_club_id_list"))

# Unire il dataframe originale con quello ottenuto al passaggio precedente sulla chiave (player_id, year)
# e calcolare la somma delle colonne yellow_cards, red_cards, goals, assists e minutes_played raggruppando sulla nuova chiave (player_id, competition_id, year, player_club_id)
df_result = df_new.join(df_grouped, ["player_id", "year"]) \
              .groupBy("player_id", "competition_id", "year", "player_club_id") \
              .agg(sum("yellow_cards").alias("yellow_cards"),
                   sum("red_cards").alias("red_cards"),
                   sum("goals").alias("goals"),
                   sum("assists").alias("assists"),
                   sum("minutes_played").alias("minutes_played")) \
              .groupBy("player_id", "year") \
              .agg(collect_set("competition_id").alias("competition_id"),
                   collect_set("player_club_id").alias("player_club_id"),
                   sum("yellow_cards").alias("yellow_cards"),
                   sum("red_cards").alias("red_cards"),
                   sum("goals").alias("goals"),
                   sum("assists").alias("assists"),
                   sum("minutes_played").alias("minutes_played"))

# Visualizza il risultato
df_result.show()

+---------+----+--------------+--------------+------------+---------+-----+-------+--------------+
|player_id|year|competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|
+---------+----+--------------+--------------+------------+---------+-----+-------+--------------+
|       26|2017|      [CL, L1]|          [16]|           0|        0|    0|      0|           181|
|      215|2014|    [ES1, CDR]|        [1084]|           0|        0|    5|      1|           985|
|      488|2014|         [GB1]|        [2288]|           0|        0|    0|      0|           109|
|      532|2014| [CL, DFB, L1]|          [27]|           0|        0|    1|      0|           216|
|      532|2016|     [DFB, L1]|          [86]|           2|        0|   13|      4|          1854|
|      532|2017|          [L1]|       [86, 3]|           1|        0|    1|      0|           573|
|      532|2020|          [L1]|          [86]|           0|        0|    0|      0|            36|
|      568

In [165]:
#TEST
print(df_result.filter(df_result.player_id == 148455).count())
df_result.filter(df_result.player_id == 148455).agg(sum(col("goals"))).show()

10
+----------+
|sum(goals)|
+----------+
|       227|
+----------+



In [166]:
df_result.filter(df_result.player_id == 148455).show()

+---------+----+--------------------+--------------+------------+---------+-----+-------+--------------+
|player_id|year|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|
+---------+----+--------------------+--------------+------------+---------+-----+-------+--------------+
|   148455|2016|  [CL, EL, IT1, CLQ]|          [12]|           3|        0|   18|     10|          3549|
|   148455|2015|[CL, FAC, CIT, EL...|[12, 430, 631]|           3|        0|   15|      5|          3351|
|   148455|2014|           [CL, GB1]|         [631]|           0|        0|    0|      0|            89|
|   148455|2017|[CL, CIT, EL, IT1...|      [12, 31]|           0|        0|   33|     14|          3831|
|   148455|2019|[CL, GBCS, FAC, G...|          [31]|           2|        0|   26|     11|          4467|
|   148455|2018|      [CL, FAC, GB1]|          [31]|           2|        0|   37|     16|          4044|
|   148455|2020|[CL, GBCS, FAC, GB1]|          [31]|   

In [170]:
df_players_stats = df_result.join(df_players, ["player_id"], how='inner')
df_players_stats.filter(df_players_stats.player_id == 148455).show()

+---------+----+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+---------------+-----------------+----------------------+----------------+-----------------+-------------+--------+------------+----+------------+-------------------+---------------------------+----------+------------------------+------------------------------------+----------+-------------+-------------+--------------------+-----------+--------------------+
|player_id|year|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|         name|current_club_id|current_club_name|country_of_citizenship|country_of_birth|    city_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|market_value_in_eur|highest_market_value_in_eur|agent_name|contract_expiration_date|current_club_domestic_competition_id|first_name|    last_name|  player_code|           image_url|last_season|                 url|
+---------+----+--------------------+-----

In [137]:
df_players_stats = df_players_stats.drop("current_club_id", "current_club_name", "city_of_birth", "market_value_in_eur", "contract_expiration_date", "agent_name", "current_club_domestic_competition_id", "image_url", "last_season", "url", "player_current_club_id")

In [138]:
df_players_stats.filter(df_players_stats.player_id == 148455).show()

+---------+----+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+---------------------------+----------+-------------+-------------+
|player_id|year|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|highest_market_value_in_eur|first_name|    last_name|  player_code|
+---------+----+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+---------------------------+----------+-------------+-------------+
|   148455|2016|  [CL, EL, IT1, CLQ]|          [12]|           3|        0|   18|     10|          3549|Mohamed Salah|                 Egypt|           Egypt|  

In [171]:
print("Number of rows: ", df_players_stats.count())

Number of rows:  72357


In [179]:
df_player_valutations.filter(df_player_valutations.player_id == 148455).show()

+----------+----------+----------+---------+---------------+-------------------+-----------------------------------+
|      date|  datetime|  dateweek|player_id|current_club_id|market_value_in_eur|player_club_domestic_competition_id|
+----------+----------+----------+---------+---------------+-------------------+-----------------------------------+
|2010-06-06|2010-06-06|2010-05-31|   148455|             31|              25000|                                GB1|
|2011-01-01|2011-01-01|2010-12-27|   148455|             31|              25000|                                GB1|
|2011-05-09|2011-05-09|2011-05-09|   148455|             31|              50000|                                GB1|
|2011-07-14|2011-07-14|2011-07-11|   148455|             31|             150000|                                GB1|
|2011-12-25|2011-12-25|2011-12-19|   148455|             31|             300000|                                GB1|
|2012-05-27|2012-05-27|2012-05-21|   148455|             31|    

In [183]:
#eliminare da df_players_valuatations le colonne datetime, dateweek, current_club_id, player_club_domestic_id
df_player_valutations = df_player_valutations.drop("datetime", "dateweek", "current_club_id", "player_club_domestic_id")
#df_player_valutations.filter(df_player_valutations.player_id == 148455).show()
#a df_players_valuatations aggiungere la colonna year che contiene solo l'anno della feature date
from pyspark.sql.functions import year
df_player_valutations_c = df_player_valutations.withColumn("year", year(df_player_valutations.date))
df_player_valutations_c.filter(df_player_valutations_c.player_id == 148455).show(30)

+----------+---------+-------------------+-----------------------------------+----+
|      date|player_id|market_value_in_eur|player_club_domestic_competition_id|year|
+----------+---------+-------------------+-----------------------------------+----+
|2010-06-06|   148455|              25000|                                GB1|2010|
|2011-01-01|   148455|              25000|                                GB1|2011|
|2011-05-09|   148455|              50000|                                GB1|2011|
|2011-07-14|   148455|             150000|                                GB1|2011|
|2011-12-25|   148455|             300000|                                GB1|2011|
|2012-05-27|   148455|            1500000|                                GB1|2012|
|2012-07-12|   148455|            2500000|                                GB1|2012|
|2012-09-05|   148455|            4000000|                                GB1|2012|
|2013-04-14|   148455|            5000000|                                GB

In [184]:
#in df_player_valutations, per ogni coppia (year,player_id) lasciare solo l'istanza con l'ultimo date dell'anno year
from pyspark.sql.functions import max
df_player_valutations_c = df_player_valutations_c.groupBy("year", "player_id").agg(max("date").alias("date")).select("year", "player_id", "date")
df_player_valutations_c.filter(df_player_valutations_c.player_id == 148455).show()

+----+---------+----------+
|year|player_id|      date|
+----+---------+----------+
|2011|   148455|2011-12-25|
|2014|   148455|2014-08-12|
|2023|   148455|2023-03-16|
|2015|   148455|2015-11-05|
|2018|   148455|2018-12-19|
|2021|   148455|2021-12-23|
|2012|   148455|2012-09-05|
|2016|   148455|2016-07-15|
|2013|   148455|2013-11-26|
|2022|   148455|2022-11-03|
|2019|   148455|2019-12-10|
|2020|   148455|2020-10-13|
|2017|   148455|2017-10-23|
|2010|   148455|2010-06-06|
+----+---------+----------+



In [144]:
#join tra df_player_valutations_c e df_player_valutations su player_id. Da df_player_valutations prendere solo la colonna market_value_in_eur
df_player_valutations = df_player_valutations.join(df_player_valutations_c, ["year", "player_id", "date"], how='inner').select("year", "player_id", "date", "market_value_in_eur")
df_player_valutations.filter(df_player_valutations.player_id == 148455).show()

+----+---------+----------+-------------------+
|year|player_id|      date|market_value_in_eur|
+----+---------+----------+-------------------+
|2010|   148455|2010-06-06|              25000|
|2011|   148455|2011-12-25|             300000|
|2012|   148455|2012-09-05|            4000000|
|2013|   148455|2013-11-26|            8500000|
|2014|   148455|2014-08-12|           13000000|
|2015|   148455|2015-11-05|           20000000|
|2016|   148455|2016-07-15|           27000000|
|2017|   148455|2017-10-23|           40000000|
|2018|   148455|2018-12-19|          150000000|
|2019|   148455|2019-12-10|          150000000|
|2020|   148455|2020-10-13|          120000000|
|2021|   148455|2021-12-23|          100000000|
|2022|   148455|2022-11-03|           80000000|
|2023|   148455|2023-03-16|           70000000|
+----+---------+----------+-------------------+



In [145]:
#merge tra df_players_stats e df_player_valutations su player_id e year
df_players_stats = df_players_stats.join(df_player_valutations, ["player_id", "year"], how='inner')
df_players_stats.show()

+---------+----+--------------+--------------+------------+---------+-----+-------+--------------+------------------+----------------------+-----------------+-------------+----------+------------------+-----+------------+---------------------------+----------+------------+------------------+----------+-------------------+
|player_id|year|competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|              name|country_of_citizenship| country_of_birth|date_of_birth|  position|      sub_position| foot|height_in_cm|highest_market_value_in_eur|first_name|   last_name|       player_code|      date|market_value_in_eur|
+---------+----+--------------+--------------+------------+---------+-----+-------+--------------+------------------+----------------------+-----------------+-------------+----------+------------------+-----+------------+---------------------------+----------+------------+------------------+----------+-------------------+
|       26|2017|      [CL, L

In [146]:
df_players_stats.filter(df_players_stats.player_id == 148455).show()

+---------+----+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+---------------------------+----------+-------------+-------------+----------+-------------------+
|player_id|year|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|highest_market_value_in_eur|first_name|    last_name|  player_code|      date|market_value_in_eur|
+---------+----+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+---------------------------+----------+-------------+-------------+----------+-------------------+
|   148455|2014|           [CL, GB1]|         [631]|           0|  

In [185]:
print("Number of rows: ", df_players_stats.count())

Number of rows:  72357


In [1]:
# a df_players_stats aggiungere la colonna partite_giocate che contiene il numero di partite giocate dalle sue squadre (player_club_id) in year;
# a df_player_stats aggiungere la colonna punti_ottenuti che contiene il numero di punti ottenuti dalle sue squadre (player_club_id) in year; (3 punti per vittoria, 1 per pareggio, 0 per sconfitta)
# a df_player_stats aggiungere la colonna media_punti che contiene la media dei punti ottenuti dalle sue squadre (player_club_id) in year; (ottenerla dalle colonne partite_giocate e punti_ottenuti)